In [10]:
import requests
import json
import time
import os
from dotenv import load_dotenv
from web3 import Web3
from web3.datastructures import AttributeDict
from hexbytes import HexBytes

In [2]:
load_dotenv()
the_graph_api_key = os.getenv('THE_GRAPH_API_KEY')
infura_api_key = os.getenv('INFURA_API_KEY')

aave_url = f"https://gateway.thegraph.com/api/{the_graph_api_key}/subgraphs/id/8EBbn3tNayccBZrnW9ae6Q4NLHfVEcozvkB3YAp5Qatr"
infura_mainnet_url = f'https://mainnet.infura.io/v3/{infura_api_key}'

web3 = Web3(Web3.HTTPProvider(infura_mainnet_url))

In [ ]:
logs = web3.eth.get_logs({
    'fromBlock': 17000001,
    'toBlock': 21336719,
    'topics': [
        '0xe8d51c8e11bd570db1734c8ec775785330e77007feed45c43b608ef33ff914bd'
    ]
})

In [ ]:
def convert_to_serializable(obj):
    if isinstance(obj, AttributeDict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, HexBytes):
        return obj.hex()
    elif isinstance(obj, list):
        return [convert_to_serializable(i) for i in obj]
    else:
        return obj

delegations = [convert_to_serializable(log) for log in logs]

In [ ]:
delegations = sorted(delegations, key=lambda x: x['blockNumber'], reverse=True)

fields_to_remove = ['blockHash', 'logIndex', 'removed', 'transactionHash', 'transactionIndex']

for delegation in delegations:
    for field in fields_to_remove:
        delegation.pop(field, None)
        
def decode_hex(hex_string):
    return web3.to_int(hexstr=hex_string)

for delegation in delegations:
    delegation['type'] = decode_hex(delegation['data'])
    delegation['from'] = web3.to_checksum_address('0x' + delegation['topics'][1][-40:])
    delegation['to'] = web3.to_checksum_address('0x' + delegation['topics'][2][-40:])
    delegation.pop('data')
    delegation.pop('topics')

In [58]:
addresses = ['0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9', '0x4da27a545c0c5B758a6BA100e3a049001de870f5', '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9', '0xa1116930326D21fB917d5A27F1E9943A9595fb47']
filtered_delegations = [entry for entry in delegations if entry['address'] in addresses]

In [ ]:
web3.eth.get_block(filtered_delegations[0]['blockNumber'])['timestamp']

for entry in filtered_delegations:
    entry['timestamp'] = web3.eth.get_block(entry['blockNumber'])['timestamp']

In [67]:
for entry in filtered_delegations:
    entry.pop('blockNumber')

In [68]:
with open('../data/proposals/aave/aave_2.json', 'w') as f:
    json.dump(filtered_delegations, f, indent=4)

In [6]:
def retrieve_delegations():
    skip = 0
    all_data = []
    
    while True:
        body = f"""
        {{
            delegateChanges(
                orderBy: blockTimestamp
                orderDirection: desc
                first: 1000
                skip: {skip}
            ) {{
            tokenAddress
            delegator
            delegate
            blockTimestamp
            }}
        }}
        """

        response = requests.post(url=aave_url, json={"query": body})
        
        if response.status_code != 200:
            raise Exception(f"Query failed with {response.status_code}: {response.text}")
        
        data = response.json().get("data", {}).get("delegateChanges", [])
        
        if not data:
            break
        
        all_data.extend(data)
        skip += 1000
        
    with open(f'../data/proposals/aave/aave_delegations.json', 'w', encoding='utf8') as outfile:
        json.dump(all_data, outfile, ensure_ascii=False, indent=4)
    

In [7]:
retrieve_delegations()

In [6]:
def retrieve_votes(proposal):
    body = f"""
    {{
        votes(
            orderBy: weight
            orderDirection: desc
            first: 1000
            where: {{
                proposal_: {{
                    id: "{proposal}"
                }}
            }}
        ) {{
        choice
        id
        weight
        }}
    }}
    """

    response = requests.post(url=aave_url, json={"query": body})
    
    if response.status_code == 200:
        return response.json().get("data", {}).get("votes", [])
    else:
        return []

In [7]:
with open ('../data/proposals/aave/aave_v2_on_chain_proposals_1_year.json', 'r', encoding='utf8') as file:
    aave_v2_proposals = json.load(file)['data']['proposals']

In [8]:
all_votes = []

for proposal in aave_v2_proposals:
    proposal_id = proposal['id']
    votes = retrieve_votes(proposal_id)
    
    all_votes.append({
        "proposal_id": proposal_id,
        "votes": votes
    })
    time.sleep(1)
    
with open ('../data/proposals/aave/aave_v2_on_chain_votes_1.json', 'w', encoding='utf8') as outfile:
    json.dump(all_votes, outfile, ensure_ascii=False)